In [1]:
from tristo import tables_to_db, scrape_pages_WVU, update_file_info, extract_tables
import os
os.getcwd()

c:\Users\Leon\anaconda3\envs\tristo\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Leon\anaconda3\envs\tristo\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\Leon\anaconda3\envs\tristo\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


'g:\\UBA_WIMI\\1_TriSto\\4_Skript_v2\\Tristo2'

In [ ]:
os.chdir('..')

In [2]:
scrape_pages_WVU('Trinkwasserqualität OR Wasseranalyse', n_res=3, wait=30)

c:\Users\Leon\anaconda3\envs\tristo\lib\site-packages\sqlmodel\orm\session.py:60: SAWarning: Class SelectOfScalar will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  results = super().execute(



(None, None, None)


In [3]:
update_file_info('site - Trinkwasserqualität OR Wasseranalyse')

5485 of 5485: "46535079328f842bfa0a8be88d24d1e8de050c96/www.pdf"


In [4]:
extract_tables('site - Trinkwasserqualität OR Wasseranalyse')

31 of 31: "88ef565b5a71bf135cb66cb087c8155a6c114f57/2019-wasseranalyse-swg-hb-hochzone.pdf"


In [5]:
tables_to_db('site - Trinkwasserqualität OR Wasseranalyse')

In [3]:
import sqlite3
import os
print(os.getcwd())
os.chdir('site - Trinkwasserqualität OR Wasseranalyse')
import re

def sub(text, pat, new):
    return re.sub(pat, new, text.replace('\n', ''), flags=re.DOTALL|re.IGNORECASE)

def regexp(text, pat):
    return re.fullmatch(pat,text.replace('\n', ''), flags=re.DOTALL|re.IGNORECASE) is not None

def search(text, pat):
    return re.search(pat,text.replace('\n', ''), flags=re.DOTALL|re.IGNORECASE) is not None


g:\UBA_WIMI\1_TriSto\4_Skript_v2\Tristo2


In [8]:
from sqlalchemy import create_engine, select
from sqlalchemy.orm import Session

In [6]:
engine = create_engine("sqlite:///db.db", echo= True)

In [12]:
with Session(engine) as session:
    print(session.identity_map())

TypeError: 'WeakInstanceDict' object is not callable

In [9]:
# update params with unique matches
with sqlite3.connect('db.db') as con:
    con.create_function('sub', 3, sub)
    con.create_function('regexp', 2, regexp)
    con.create_function('search', 2, search)
    bl = 'SELECT DISTINCT d.param, rebl.regex FROM (SELECT DISTINCT param FROM data) d CROSS JOIN reblacklist rebl WHERE SEARCH(d.param, rebl.regex)'
    nobl = 'SELECT DISTINCT d.param FROM (SELECT DISTINCT param FROM data WHERE param_id IS NULL) d CROSS JOIN reblacklist rebl GROUP BY d.param HAVING SUM(SEARCH(d.param, rebl.regex))=0'
    sql= f'SELECT d.param from ({nobl}) d CROSS JOIN reparam re WHERE REGEXP(d.param, re.regex) GROUP BY d.param HAVING SUM(REGEXP(d.param, re.regex))=1'
    sql2 = f'SELECT DISTINCT d.param, re.idx from ({sql}) d CROSS JOIN reparam re WHERE REGEXP(d.param, re.regex)'
    sql3 = f'UPDATE data SET param_id=d.idx FROM ({sql2}) d WHERE d.param = data.param'
    for row in con.execute(sql3):
        print(row)
    con.commit()

In [1]:
# print params with multiple matches
with sqlite3.connect('db.db') as con:
    con.create_function('sub', 3, sub)
    con.create_function('regexp', 2, regexp)
    con.create_function('search', 2, search)
    nobl = 'SELECT DISTINCT d.param FROM (SELECT DISTINCT param FROM data) d CROSS JOIN reblacklist rebl GROUP BY d.param HAVING SUM(SEARCH(d.param, rebl.regex))=0'
    sql= f'SELECT d.param from ({nobl}) d CROSS JOIN reparam re WHERE REGEXP(d.param, re.regex) GROUP BY d.param HAVING SUM(REGEXP(d.param, re.regex))>1 AND COUNT(DISTINCT re.param)>1 '
    sql2 = f'SELECT DISTINCT d.param, re.regex, re.param from ({sql}) d CROSS JOIN reparam re WHERE REGEXP(d.param, re.regex)'
    for row in con.execute(sql2):
        print(row)

NameError: name 'sqlite3' is not defined

In [4]:
with sqlite3.connect('db.db') as con:
    sql= 'SELECT distinct regex from reparam'
    for row, in con.execute(sql):
        print(row)
        regexp('a',row)

.*Acrylamid.*
Aluminium
Antimon
Arsen
.*enzo?.*a.*pyre.*
.*Benzene?|B?enzol
Pb
Bor
.*Bromat.*
Cadm?ium.*
.*[kc]al?[kc][il]t.*l.{,2}s.*
Ch?loride?
Chrom
.*Ch?lostr|.*perfr.*
.*c[oa]..form.*
E.sen|iron
.*E.t?ero([ck]o)?[kc]{2}.*
Epichlorhydrin
.*e[\.\s]+col[il].*
.*f.{1,2}rbung.*
Fluoride?
Geruc.*(qual|sens).*
Geschmac?k.*
Kupfer
Mangan
Natrium
Nickel
(gesa.*)?.*org[\.].*kohlen.*
.*[Oo]x[iy]dier.*
.*Pflanzen(schutz|behan.*)mittel.+einzel.*
.*Pflanzen.+(ins|ges|Summe).*
.*pol[iy][cz]y[ck].*
Queck.?silber?|mercury
.*R(ado)?n.*222.*
Richtdosis
Se.en.*(se)?
Sulfat
.*([Tt]ri|[Tt]etra).+([Tt]ri|[Tt]etra).*(chlorethen)?.*
.*[Tt]rihalo.*[Mm]eth?ane.*|.*THM.*
Tritium
.*T.{1,3}bun.*|.*NTU.*
Uran
.*Viny.c..orid.*|Ch.orethen
ph[\s~-]*wert|Wasserstoffionen.*Konz.*
Alachlor
Anthracen
Atrazin
Bromierte Diphenylether
Cadmium und Cadmiumverbindungen (je nach Wasserhärteklasse)
Tetrachlorkohlenstoff
C10-13-Chloralkane
Chlorfenvinphos
Chlorpyrifos-Ethyl
Cyclodien Pestizide:  Aldrin  Dieldrin  Endrin  Isodr

In [1]:
from tristo.database import OpenDB, Data, REParam
from tristo.complements import TO_REPLACE, LIMITS
from sqlmodel import select
import numpy as np



In [ ]:
TO_REPLACE['index']

In [ ]:
exp=[]
rows = 0
for key, values in LIMITS.items():
    rows+=values.index.size
    if (col:='to_replace') in values:
        for param, regex in values[col].iteritems():
            regex = regex if type(regex)!=float else param
            exp.append(REParam(param=param, regex=regex))
    else:
        for param in values.index:
            exp.append(REParam(param=param, regex=param))
print(rows)


In [2]:
import os
os.chdir('site - Trinkwasserqualität OR Wasseranalyse')
with OpenDB().session() as session:
    pass
    # exp=[]
    # rows = 0
    # for key, values in LIMITS.items():
    #     rows+=values.index.size
    #     if (col:='to_replace') in values:
    #         for param, regex in values[col].iteritems():
    #             regex = regex if type(regex)!=float else param
    #             exp.append(REParam(param=param, regex=regex))
    #     else:
    #         for param in values.index:
    #             exp.append(REParam(param=param, regex=param))
    # print(rows)
    # session.add_all(exp)
    # session.commit()


In [28]:
text= 'KBE/100ml\nKoloniezahl bei 20°C'
pat = '.*KB.*(?P<ph>\d[,\.\s]{1,2}\d).*'
regexp(text, pat)

False

In [11]:
re.sub('.*leitf.*(?P<T>\d{2}).*', 'Elektrische Leitfähigkeit bei \g<T> °C', text, flags=re.DOTALL|re.IGNORECASE)


'Elektrische Leitfähigkeit bei 72 °C'

In [10]:
param= 'Acridin--9--carbonsäure'
print(f"{param.replace('.',',').replace('--','-')}\t{param.replace('.','.?').replace('--','.*')}")

Acridin-9-carbonsäure	Acridin.*9.*carbonsäure
